## Importing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from sklearn.model_selection import GroupKFold
from tensorflow.keras.layers import LSTM
from sklearn.metrics import roc_auc_score

## Loading Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-apr-2022/train.csv")
train.head()

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-apr-2022/test.csv")
test.head()

In [ ]:
tl = pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")
tl.head()

In [ ]:
groups = train["sequence"]
train = train.drop(["sequence", "subject", "step"], inplace=False, axis=1).values
test = test.drop(["sequence", "subject", "step"], inplace=False, axis=1).values
tl = tl["state"]
train = train.reshape(int(len(train)/60), 60, 13)
test = test.reshape(int(len(test)/60), 60, 13)

In [ ]:
model = Sequential()
model.add(LSTM(500, return_sequences=True))
model.add(LSTM(400, return_sequences=True))
model.add(LSTM(300, return_sequences=True))
model.add(LSTM(200, return_sequences=True))
model.add(Conv1D(32, 7))
model.add(MaxPooling1D())
model.add(Conv1D(64, 3))
model.add(MaxPooling1D())
model.add(Conv1D(128, 3))
model.add(GlobalMaxPooling1D())
model.add(Dense(150, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
cv_score = 0
test_preds = []
kf = GroupKFold(n_splits=5)
for fold_idx, (train_idx, valid_idx) in enumerate(kf.split(train, tl, groups.unique())):
    
    print("*"*15, f"Fold {fold_idx+1}", "*"*15)
    
    X_train, X_valid = train[train_idx], train[valid_idx]
    y_train, y_valid = tl.iloc[train_idx].values, tl.iloc[valid_idx].values
    
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100, batch_size=256, 
              callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])
    
    cv_score += roc_auc_score(y_valid, model.predict(X_valid).squeeze())
    
    test_preds.append(model.predict(test).squeeze())
    
print(cv_score/5)

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv")

submission["state"] = sum(test_preds)/5
submission.to_csv("submission.csv", index=False)
submission